In [1]:
# import packages
import pandas as pd
import json

# pip install mtgsdk
from mtgsdk import Card

In [2]:
# Import the previously scraped data, and create a master card list for adding additional data to and referencing later

# Maindeck import
maindeck_filepath = "data/maindeck_cards.csv"
maindeck_df = pd.read_csv(maindeck_filepath)

# Sideboard import
sideboard_filepath = "data/sideboard_cards.csv"
sideboard_df = pd.read_csv(sideboard_filepath)

# Concatenate only the 'card name' column from maindeck_df and sideboard_df
all_cards_df = pd.concat([maindeck_df[['card name']], sideboard_df[['card name']]], ignore_index=True)

# Keep only unique rows based on 'card name'
all_cards_df = all_cards_df.drop_duplicates(ignore_index=True)

# Add a unique id for each card
all_cards_df['card id'] = range(1, len(all_cards_df) + 1)

# Rearrange dataframe
all_cards_df = all_cards_df[['card id', 'card name']]

# Export to CSV
all_cards_df.to_csv("data/all_cards.csv", index=False)

all_cards_df


,card id,card name
0,1,Aether Spellbomb
1,2,Aether Vial
2,3,Aluren
3,4,Ancient Tomb
4,5,Animate Dead
...,...,...
2304,2305,Leyline of Anticipation
2305,2306,"Aeve, Progenitor Ooze"
2306,2307,Pyroclasm
2307,2308,Tome Scour


In [4]:
# testing api

# cardname = all_cards_df['card name'][0]
cardname = "Shivan Dragon"

try:
    cards = Card.where(name = cardname).all()
    exact_match = next((card for card in cards if card.name == cardname), cards[0])
    print(f'Card name: {exact_match.name}')
    print(f'Colors: {exact_match.colors}')
    print(f'Converted Mana Cost: {int(exact_match.cmc)}')
    print(f'Type: {exact_match.type}')
except IndexError:
    print(f"card not found - index error")
except Exception as e:
    print(f"An error occurred for card '{cardname}': {e}")

Card name: Shivan Dragon
Colors: ['R']
Converted Mana Cost: 6
Type: Creature — Dragon


In [85]:
# WARNING ------------------------
# Executing this cell will take 25+ minutes to fully collect the data.
# To see the final output, see the subsequent csv in data/card_info.csv
# Note that cards missing from the API are from the newest set. These will be manually cleaned as its a small subset.

card_info = []
for index, row in all_cards_df.iterrows():
    cardname = row['card name']
    card_id = row['card id']
    try:
        cards = Card.where(name = cardname).all()
        exact_match = next((card for card in cards if card.name == cardname), cards[0])
        card_info.append({'card id': card_id, 'card name': cardname, 'colors': exact_match.colors, 'cmc': int(exact_match.cmc), 'type': exact_match.type})
        print(f"{row['card name']} information found. Next card.")
    except:
        card_info.append({'card id': card_id,'card name': cardname, 'colors': 'card not found', 'cmc': 'card not found', 'type': 'card not found'})
        print("------------")
        print(f"!!! {cardname} not found!")
        print("------------")
print("Data pull complete!")


Aether Spellbomb information found. Next card.
Aether Vial information found. Next card.
Aluren information found. Next card.
Ancient Tomb information found. Next card.
Animate Dead information found. Next card.
Architects of Will information found. Next card.
Archon of Cruelty information found. Next card.
Atraxa, Grand Unifier information found. Next card.
Badlands information found. Next card.
Battle Cry Goblin information found. Next card.
Bayou information found. Next card.
Blast Zone information found. Next card.
Bloodbraid Marauder information found. Next card.
Bloodstained Mire information found. Next card.
Bojuka Bog information found. Next card.
Boseiju, Who Endures information found. Next card.
Brainstorm information found. Next card.
Brazen Borrower information found. Next card.
------------
!!! Broadside Bombardiers not found!
------------
Cavern Harpy information found. Next card.
Cavern of Souls information found. Next card.
Caves of Chaos Adventurer information found. N

In [86]:
# Show the additional data in a dataframe
card_info_df = pd.DataFrame(card_info)

# Store as a csv
card_info_df.to_csv("data/card_info.csv", index=False)

card_info_df

,card id,card name,colors,cmc,type
0,1,Aether Spellbomb,None,1,Artifact
1,2,Aether Vial,None,1,Artifact
2,3,Aluren,[G],4,Enchantment
3,4,Ancient Tomb,None,0,Land
4,5,Animate Dead,[B],2,Enchantment — Aura
...,...,...,...,...,...
2304,2305,Leyline of Anticipation,[U],4,Enchantment
2305,2306,"Aeve, Progenitor Ooze",[G],5,Legendary Creature — Ooze
2306,2307,Pyroclasm,[R],2,Sorcery
2307,2308,Tome Scour,[U],1,Sorcery


In [87]:
# Cards missing from the API are from the newest set. Manually cleaned up the 131 missing records for completeness. 
# Re-import the completed csv
# Also has the benefit of not re-running the API call process

file = "data/card_info_complete.csv"
card_info_complete_df = pd.read_csv(file)

card_info_complete_df

,card id,card name,colors,cmc,type
0,1,Aether Spellbomb,NaN,1,Artifact
1,2,Aether Vial,NaN,1,Artifact
2,3,Aluren,['G'],4,Enchantment
3,4,Ancient Tomb,NaN,0,Land
4,5,Animate Dead,['B'],2,Enchantment — Aura
...,...,...,...,...,...
2304,2305,Leyline of Anticipation,['U'],4,Enchantment
2305,2306,"Aeve, Progenitor Ooze",['G'],5,Legendary Creature — Ooze
2306,2307,Pyroclasm,['R'],2,Sorcery
2307,2308,Tome Scour,['U'],1,Sorcery


In [88]:

# Split the type into type and subtype, and store the results as a list for each for parsing for graphs later.
# Convert 'type' column to string
card_info_complete_df['type'] = card_info_complete_df['type'].astype(str)

# Create new 'type' and 'subtype' columns
split_df = card_info_complete_df['type'].apply(lambda x: pd.Series(x.split(' — ') if ' — ' in x else [x, None]))

# Rename the resulting columns
split_df.columns = ['type', 'subtype']

# Drop the original 'type' column
card_info_complete_df.drop('type', axis=1, inplace=True)

# Concatenate the new columns with the original DataFrame
card_info_complete_df = pd.concat([card_info_complete_df, split_df], axis=1)

# Covert new columns to lists
card_info_complete_df['type'] = card_info_complete_df['type'].apply(lambda x: [f"'{item}'" for item in x.split()] if x else None)
card_info_complete_df['subtype'] = card_info_complete_df['subtype'].apply(lambda x: [f"'{item}'" for item in x.split()] if x else None)

# Not sure if this is needed yet for later SQL - revisit
# Function to convert "None" to an empty list
# def transform_value(value):
#     return [] if value == 'None' or value == None else value

# card_info_complete_df = card_info_complete_df.applymap(transform_value)

card_info_complete_df

,card id,card name,colors,cmc,type,subtype
0,1,Aether Spellbomb,NaN,1,['Artifact'],None
1,2,Aether Vial,NaN,1,['Artifact'],None
2,3,Aluren,['G'],4,['Enchantment'],None
3,4,Ancient Tomb,NaN,0,['Land'],None
4,5,Animate Dead,['B'],2,['Enchantment'],['Aura']
...,...,...,...,...,...,...
2304,2305,Leyline of Anticipation,['U'],4,['Enchantment'],None
2305,2306,"Aeve, Progenitor Ooze",['G'],5,"['Legendary', 'Creature']",['Ooze']
2306,2307,Pyroclasm,['R'],2,['Sorcery'],None
2307,2308,Tome Scour,['U'],1,['Sorcery'],None


#### Include Pricing Information for Each card
* To capture finance data, I collected a set of text files that have daily pricing information for all of January for every card from https://www.goatbots.com/download-prices. 
* I wrote a script (\scripts\combine_pricing_data_files_script.py) to combine each text file and include its date as part of a master JSON file
* There are typically many variations per card. To simplify, I'll capture the lowest price of all versions to associate with the tournament decks

In [145]:
# Import the definition file as a dataframe, 
card_definition_file = "data/prices/definitions/card-definitions.json"
card_definitions_df = pd.read_json(card_definition_file, orient='records').T
card_definitions_df = card_definitions_df.rename_axis('MTGO_id').reset_index()


In [146]:
# Do some naming cleanup to match other files 
card_definitions_df['name'] = card_definitions_df['name'].str.replace('"Name Sticker"', '_____')
card_definitions_df['name'] = card_definitions_df['name'].str.replace('/', ' // ')

In [147]:
# Filter to only card names that are in tournament lists (removes 70,000 rows!)
filtered_card_definitions_df = card_definitions_df[card_definitions_df['name'].isin(card_info_complete_df['card name'])]

filtered_card_definitions_df

,MTGO_id,name,cardset,rarity,foil
1,121150,Watery Grave,RVR,Rare,1
2,121149,Watery Grave,RVR,Rare,0
3,121148,Temple Garden,RVR,Rare,1
4,121147,Temple Garden,RVR,Rare,0
5,121146,Stomping Ground,RVR,Rare,1
...,...,...,...,...,...
78529,232,Plains,PRM,Common,1
78530,231,Plains,PRM,Common,0
78531,226,Zuran Orb,PRM,Uncommon,1
78532,225,Zuran Orb,PRM,Uncommon,0


In [148]:
# Import pricing information for a single day (most recent at the time data was captured, Jan 25th)
# We are doing this because we want to find the cheapest version of each tournament played card for tracking over time
# This will then let us tie the pricing ID to our card ID
sample_pricing_file = "data/prices/price-history-2024-01-25.json"
sample_pricing_df = pd.read_json(sample_pricing_file, orient='index')
sample_pricing_df = sample_pricing_df.rename_axis('MTGO_id').reset_index()
sample_pricing_df.columns = ['MTGO_id', 'price']
sample_pricing_df

,MTGO_id,price
0,121151,2.240
1,121150,0.490
2,121149,0.240
3,121148,0.700
4,121147,1.080
...,...,...
78568,19,0.007
78569,18,0.070
78570,14,0.007
78571,13,0.200


In [149]:
# Merge the dataframes to get prices for selected card.
cards_merged_df = pd.merge(filtered_card_definitions_df, sample_pricing_df, on='MTGO_id', how='inner')

# Filter for the lowest price for each unique card name
min_price_indices = cards_merged_df.groupby('name')['price'].idxmin()
filtered_merged_df = cards_merged_df.loc[min_price_indices]
filtered_merged_df = filtered_merged_df.rename(columns={'name': 'card name'})

filtered_merged_df

,MTGO_id,card name,cardset,rarity,foil,price
4334,90371,Abiding Grace,MH2,Uncommon,0,0.003
3034,101420,Aboleth Spawn,CLB,Rare,0,0.710
390,118074,Abrade,LCI,Common,0,0.002
9947,46497,Abrupt Decay,RTR,Rare,0,0.060
6570,71302,Absorb,RNA,Rare,0,0.030
...,...,...,...,...,...,...
8437,58401,Zulaport Cutthroat,BFZ,Uncommon,0,0.004
4162,91027,Zuran Orb,MH2,Uncommon,0,0.003
8754,56050,Zurgo Bellstriker,DTK,Rare,0,0.006
847,115722,_____ Goblin,UNF,Common,0,4.810


In [150]:
# Now we can finally relate our original unique card ids to the MTGO_id from the pricing sheets for our DB.
card_info_master_df = pd.merge(card_info_complete_df,filtered_merged_df, on='card name', how='inner')
card_info_master_df = card_info_master_df.rename(columns={'price': 'latest price'})

card_info_master_df

,card id,card name,colors,cmc,type,subtype,MTGO_id,cardset,rarity,foil,latest price
0,1,Aether Spellbomb,NaN,1,['Artifact'],None,48916,MMA,Common,0,0.002
1,2,Aether Vial,NaN,1,['Artifact'],None,37941,TD0,Uncommon,0,1.990
2,3,Aluren,['G'],4,['Enchantment'],None,56616,TPR,Rare,0,0.500
3,4,Ancient Tomb,NaN,0,['Land'],None,70056,UMA,Rare,1,16.930
4,5,Animate Dead,['B'],2,['Enchantment'],['Aura'],43142,PD3,Uncommon,1,0.009
...,...,...,...,...,...,...,...,...,...,...,...
2303,2305,Leyline of Anticipation,['U'],4,['Enchantment'],None,37368,M11,Rare,0,0.006
2304,2306,"Aeve, Progenitor Ooze",['G'],5,"['Legendary', 'Creature']",['Ooze'],91091,MH2,Rare,0,0.006
2305,2307,Pyroclasm,['R'],2,['Sorcery'],None,67206,A25,Uncommon,0,0.003
2306,2308,Tome Scour,['U'],1,['Sorcery'],None,49111,M14,Common,0,0.002


In [96]:
# Store the data as a final csv
card_info_master_df.to_csv("data/card_info_master.csv", index=False)

#### Final Cleanup for SQL DB Ingestion

In [189]:
# Events Table
starter_file = "data/league_information.csv"

# Column Renames
events_df = pd.read_csv(starter_file)
events_df = events_df.rename(columns={'event id': 'EventID', 'format': 'Format', 'event type': 'Type', 'link': 'Link', 'date': 'EventDate'})

# Export to final folder
events_df.to_csv("data/final/Events.csv", index=False)
events_df.to_json("data/final/Events.json", orient = "records")

events_df.head()

,EventID,Format,Type,Link,EventDate
0,3,Legacy,League,https://www.mtgo.com/decklist/legacy-league-20...,2024-01-27
1,4,Modern,League,https://www.mtgo.com/decklist/modern-league-20...,2024-01-27
2,5,Pauper,League,https://www.mtgo.com/decklist/pauper-league-20...,2024-01-27
3,6,Pioneer,League,https://www.mtgo.com/decklist/pioneer-league-2...,2024-01-27
4,7,Standard,League,https://www.mtgo.com/decklist/standard-league-...,2024-01-27


In [190]:
# Maindeck Table

# Bring in card ID info
maindeck_merge_df = pd.merge(maindeck_df, all_cards_df, on = "card name", how='inner')

# Column renames
maindeck_merge_df = maindeck_merge_df.rename(columns={'event id': 'EventID', 'card id': 'CardID', 'copies': 'Copies'})

# Drop card name from this table, rearrange table
maindeck_merge_df = maindeck_merge_df[['EventID', 'CardID', 'Copies']]

# Export to final folder
maindeck_merge_df.to_csv("data/final/Maindeck.csv", index=False)
maindeck_merge_df.to_json("data/final/Maindeck.json", orient = "records")

maindeck_merge_df.head()

,EventID,CardID,Copies
0,3,1,2
1,15,1,1
2,16,1,1
3,20,1,1
4,28,1,1


In [191]:
# Sideboard Table

# Bring in card ID info
sideboard_merge_df = pd.merge(sideboard_df, all_cards_df, on = "card name", how='inner')

# Column renames
sideboard_merge_df = sideboard_merge_df.rename(columns={'event id': 'EventID', 'card id': 'CardID', 'copies': 'Copies'})

# Drop card name from this table, rearrange table
sideboard_merge_df = sideboard_merge_df[['EventID', 'CardID', 'Copies']]

# Export to final folder
sideboard_merge_df.to_csv("data/final/Sideboard.csv", index=False)
sideboard_merge_df.to_json("data/final/Sideboard.json", orient = "records")

sideboard_merge_df.head()

,EventID,CardID,Copies
0,3,1917,1
1,243,1917,1
2,3,1136,2
3,18,1136,1
4,27,1136,4


In [192]:
# Cards Table

# Column Renames
card_final_df = card_info_master_df.rename(columns={'card id': 'CardID', 'card name': 'Name', 'colors': 'Colors', 'cmc': 'CMC', 'type': 'Type', 'subtype': 'Subtype', 'cardset': 'Set', 'rarity': 'Rarity', 'MTGO_id': 'MTGO_ID'})

# Drop foil and latest price from this table, rearrange table
card_final_df = card_final_df[['CardID', 'Name', 'Colors', 'CMC', 'Type', 'Subtype', 'Set', 'Rarity', 'MTGO_ID']]

# Append a row caught in QA (rather than re-run hours of scrapes)
new_row_df = pd.DataFrame({'CardID': 2265, 'Name': "Altar Of Dementia", 'Colors': None, 'CMC': 2, 'Type': ['Artifact'], 'Subtype': None, 'Set': 'BRR', 'Rarity': 'Mythic', 'MTGO_ID':104425})
new_row_df['Type'] = new_row_df['Type'].apply(lambda x: [x] if not isinstance(x, list) else x)
card_final_df = pd.concat([card_final_df, new_row_df], ignore_index=True)

card_final_df.head()

,CardID,Name,Colors,CMC,Type,Subtype,Set,Rarity,MTGO_ID
0,1,Aether Spellbomb,NaN,1,['Artifact'],None,MMA,Common,48916
1,2,Aether Vial,NaN,1,['Artifact'],None,TD0,Uncommon,37941
2,3,Aluren,['G'],4,['Enchantment'],None,TPR,Rare,56616
3,4,Ancient Tomb,NaN,0,['Land'],None,UMA,Rare,70056
4,5,Animate Dead,['B'],2,['Enchantment'],['Aura'],PD3,Uncommon,43142


In [193]:
# Prices Table

# Filepath
all_prices_file = "data/combined_pricing_data.json"

# Load the combined pricing data from JSON
with open(all_prices_file, 'r') as file:
    data = json.load(file)

# Initialize an empty list to store the dictionaries
rows = []

# Iterate through each entry in the JSON file
for entry in data:
    date = entry['date']
    prices = entry['price']

    # Iterate through each price dictionary
    for price_dict in prices:
        for mtgo_id, price in price_dict.items():
            # Create a dictionary for each row
            row = {'Date': date, 'MTGO_ID': mtgo_id, 'Price': price}

            # Append the row to the list
            rows.append(row)

# Create a DataFrame from the list of dictionaries
all_prices_df = pd.DataFrame(rows)

# Update dtype for merge
all_prices_df['MTGO_ID'] = all_prices_df['MTGO_ID'].astype('int64')

# Merge on MTGO_ID and swap to CardID in this table
all_prices_df = pd.merge(all_prices_df, card_final_df, on = "MTGO_ID", how = "inner")

# Drop card name from this table, rearrange table
all_prices_df = all_prices_df[['Date', 'CardID', 'Price']]

# Export to final folder
all_prices_df.to_csv("data/final/Prices.csv", index=False)
all_prices_df.to_json("data/final/Prices.json", orient = "records")

all_prices_df


,Date,CardID,Price
0,2024-01-01,1812,0.09
1,2024-01-02,1812,0.10
2,2024-01-03,1812,0.12
3,2024-01-04,1812,0.12
4,2024-01-05,1812,0.14
...,...,...,...
62082,2024-01-23,2266,0.20
62083,2024-01-24,2266,0.20
62084,2024-01-25,2266,0.20
62085,2024-01-26,2266,0.20


In [194]:
# Drop MTGO_ID since it was dropped from prices table
card_final_df = card_final_df[['CardID', 'Name', 'Colors', 'CMC', 'Type', 'Subtype', 'Set', 'Rarity']]

# Export to final folder
card_final_df.to_csv("data/final/Cards.csv", index=False)
card_final_df.to_json("data/final/Cards.json", orient = "records")

card_final_df.head()

,CardID,Name,Colors,CMC,Type,Subtype,Set,Rarity
0,1,Aether Spellbomb,NaN,1,['Artifact'],None,MMA,Common
1,2,Aether Vial,NaN,1,['Artifact'],None,TD0,Uncommon
2,3,Aluren,['G'],4,['Enchantment'],None,TPR,Rare
3,4,Ancient Tomb,NaN,0,['Land'],None,UMA,Rare
4,5,Animate Dead,['B'],2,['Enchantment'],['Aura'],PD3,Uncommon
